# 🧠 RAG-based Chatbot for Fashion Forward Hub  
*Powered by Google Gemini API + Weaviate Vector Database*


This notebook shows how I built a **Retrieval-Augmented Generation (RAG)** chatbot for an online clothing store, using:

- **LLM routing** – automatically decide whether a user’s query is an FAQ (refund policy, delivery, etc.) or a Product query (e.g. “cheap blue dresses for summer”) and route it to different logic.
- **Conditional parameter setting** – adjust LLM parameters (temperature, top_p, max_tokens, model) depending on the query type, so the model is more factual for FAQs and more creative/detail-rich for product recommendations.
- **Structured JSON responses** – ask the LLM to output **JSON**, then parse it into filters (e.g. gender, color, category, price range) that can be passed to Weaviate.
- **Contextual retrieval with Weaviate** – use embeddings and metadata filters to find the most relevant products.
- **Chatbot orchestration** – wrap everything into a function / class that the UI (chat widget) can call.

The goal is to demonstrate not just basic LLM usage, but **how to combine routing, parameter control, retrieval, and structured outputs** into a practical assistant for an e-commerce use case.


## ⚙️ Step 1. Environment Setup
Import required libraries, configure API keys, and connect to Weaviate.


In [1]:
import json
from weaviate.classes.query import Filter
import weaviate
import joblib

In [2]:
from utils import (
    generate_with_single_input,
    generate_with_multiple_input,
    generate_params_dict,
    generate_embedding,
    call_llm_with_context,
    _validate_and_prepare_config
)

<a id='2'></a>
## 2 - Understanding Fashion Forward Hub data schema
---
In this section, you will understand how the data is stored in Fashion Forward hub databases. 

There are two databases:

- Product database: Contains the products and their information.
- FAQ database: Contains the FAQ data.

<a id='2-1'></a>
### 2.1 Products Database

Let's explore the products database that Fashion Forward Hub has available. To make it easier to understand, let's load it as a list of JSON files first.

In [3]:
# Loading products data
PRODUCTS_DATA = joblib.load('dataset/clothes_json.joblib')

In [4]:
len(PRODUCTS_DATA)

44424

In [5]:
# Let's get one example
PRODUCTS_DATA[0]

{'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Shirts',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011.0,
 'usage': 'Casual',
 'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
 'price': 67,
 'product_id': 15970}

The features each product has are:

- **Gender:** Target audience for the product, such as "Men," "Women," or "Unisex."
- **Master Category:** Broad classification like "Apparel" or "Footwear."
- **Sub Category:** Specific category within a master category, such as "Topwear."
- **Article Type:** Exact type of product, e.g., "Shirts" or "Jackets."
- **Base Colour:** Main color of the product, important for customer choice.
- **Season:** Intended season for the product, e.g., "Summer" or "Winter."
- **Year:** Year of release or collection.
- **Usage:** Intended use or occasion, like "Casual" or "Formal."
- **Product Display Name:** Descriptive name used in marketing.
- **Price:** Cost of the product.
- **Product ID:** Unique identifier for managing and tracking inventory.

<a id='2-2'></a>
### 2.2 FAQ Database

Now let's load the FAQ database. And explore it.

In [6]:
FAQ = joblib.load("dataset/faq.joblib")

In [8]:
len(FAQ)

25

In [9]:
# Get some example
FAQ[:2]

[{'question': 'What are your store hours?',
  'answer': 'Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.',
  'type': 'general information'},
 {'question': 'Where is Fashion Forward Hub located?',
  'answer': 'Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State.',
  'type': 'general information'}]

So the FAQs are in a list with dictionaries containing `question`, `answer` and `type`. In this assignment you will work with the FAQ as a hardcoded string into a prompt, so you won't need to have a collection for querying on it. 

<a id='3'></a>
## 3. LLM Task Routing (FAQ vs Product)

The chatbot needs to behave differently depending on the type of user query:

- **FAQ queries** → “Do you offer refunds?”, “What are the shipping options?”
- **Product queries** → “Show me cheap blue jeans for women”, “I need a black dress for a wedding”

Instead of hard-coding rules, I use the LLM itself as a **router**:

1. A dedicated prompt asks the LLM to label the query as `"FAQ"` or `"Product"`, using a few examples.
2. The helper function `check_if_faq_or_product(query)` sends this routing prompt and returns the label.
3. A second function (router) looks at this label and:
   - builds the right prompt,
   - sets suitable parameters (e.g. lower temperature for FAQs),
   - and returns a `kwargs` dictionary that the rest of the system uses.

This pattern is called **LLM routing** and lets us plug in new “paths” in the future (e.g. `"SmallTalk"`, `"OrderStatus"`, etc.) without changing the whole codebase.



#### 3.1.1 Conditional LLM Parameter Setting

The notebook uses a helper function `generate_params_dict(...)` to centralize all the parameters passed to the LLM:

- `prompt` – the final instruction (including system text + user query + context)
- `temperature` – controls randomness / creativity
- `top_p` – nucleus sampling
- `max_tokens` – output length
- `model` – which Gemini model to use

Instead of calling the LLM directly with many separate arguments, I build a single **kwargs dictionary**:

```python
{
  "prompt": "...",
  "role": "user" or "assistant",
  "temperature": 0.2,
  "top_p": 0.8,
  "max_tokens": 300,
  "model": "gemini-2.0-flash"
}


In [10]:
# An output example is
kwargs = generate_params_dict("Solve x^2 - 1 = 0", temperature = 1.2, top_p = 0.2, max_tokens = 50)
print(kwargs)

{'prompt': 'Solve x^2 - 1 = 0', 'role': 'user', 'temperature': 1.2, 'top_p': 0.2, 'max_tokens': 50, 'model': 'models/gemini-2.0-flash'}


In [11]:
# Generating 
response = generate_with_single_input(**kwargs)
print(response['content'])

We are asked to solve the equation $x^2 - 1 = 0$.
We can solve this equation by factoring the left side as a difference of squares:
$$x^2 - 1 = x^2 - 1


 #### 3.1.2 The helper function `check_if_faq_or_product(query)` sends this routing prompt and returns the label.

In [12]:

def check_if_faq_or_product(query: str) -> str:
    """
    Determines whether a given instruction prompt is related to a frequently asked question (FAQ) or a product inquiry.

    Parameters:
    - query (str): The instruction or query to be labeled as either FAQ or product-related.

    Returns:
    - str: The label 'FAQ' if the prompt is classified as a frequently asked question, 'Product' if it relates to product information, or
      None if the label is inconclusive.
    """

    # Set the hardcoded prompt. Remember to include the query, clear instructions (explicitly tell the LLM to return FAQ or Product)
    # Include examples of question / desired label pairs.

    prompt = f"""
    whether a given instruction prompt is related to a frequently asked question (FAQ) or a product inquiry..
    - FAQ queries: These are related to requet asked questions.
    - Product queries: These pertain to clothing and fashion, including items like shirts, dresses, shoes, accessories, and jewelry.
    Examples:

    1. Query: “Is there a refund for incorrectly bought clothes? ” Expected answer: FAQ
    2. Query: “Tell me about the cheapest T-shirts that you have” Expected answer: Product
    3. Query: “Do you have blue T-shirts under 100 dollars??” Expected answer: Product
    4. Query: “I bought a T-shirt and I didn't like it. How can I get a refund?” Expected answer: FAQ

    Query: {query}

    Instructions: Respond with “FAQ” if the query pertains to FAQ or “Product” if it pertains to Products.
    Answer ONLY with one of the two words: FAQ or Product.
    """

    # Get the kwargs dictionary to call the LLM, with PROMPT as prompt, low temperature (0.3 - 0.5)
    # The function call is generate_params_dict, pass the PROMPT and the correct temperature
    kwargs = generate_params_dict(prompt, temperature=0.3, max_tokens=1)

    # Call generate_with_single_input with **kwargs
    response = generate_with_single_input(**kwargs)
    # Get the label by accessing the 'content' key of the response dictionary

    label = response.get("content")

    ### END CODE HERE ###
    
    return label.strip()


In [13]:
label = check_if_faq_or_product("Do you have sunglasses for children?")
print("Label:", repr(label))


Label: 'Product'


In [14]:
kwargs = generate_params_dict("Do you have sunglasses for children?", temperature = 0, top_p = 0.2, max_tokens = 50)
resp = generate_with_single_input(**kwargs)
print(resp)


{'role': 'assistant', 'content': "As an AI, I don't physically have sunglasses to sell. I am an information resource.\n\nHowever, I can help you find sunglasses for children! To give you the best recommendations, I need a little more information:\n\n*   **", 'meta': {'finish_reasons': [<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>]}}


In [15]:
print(kwargs)

{'prompt': 'Do you have sunglasses for children?', 'role': 'user', 'temperature': 0, 'top_p': 0.2, 'max_tokens': 50, 'model': 'models/gemini-2.0-flash'}


In [16]:
label = check_if_faq_or_product("Do you offer gift wrapping?")
print("Label:", repr(label))

Label: 'FAQ'


In [17]:
queries = ['What is your return policy?', 
           'Give me three examples of blue T-shirts you have available.', 
           'How can I contact the user support?', 
           'Do you have blue Dresses?',
           'Create a look suitable for a wedding party happening during dawn.']

for query in queries:
    response = check_if_faq_or_product(query)
    label = response
    print(f"Query: {query} Label: {label}")

Query: What is your return policy? Label: FAQ
Query: Give me three examples of blue T-shirts you have available. Label: Product
Query: How can I contact the user support? Label: FAQ
Query: Do you have blue Dresses? Label: Product
Query: Create a look suitable for a wedding party happening during dawn. Label: Product


<a id='3-2'></a>
### 3.2 Answering a FAQ question

Now that we have a method to decide whether a query is for FAQ or Product, we will create another function to answer a FAQ question.

This function also needs a hardcoded prompt and the FAQ question and answer pairs. For that, you will create a FAQ layout with these pairs. 

First, let's recall how the FAQ JSON is.

In [18]:
# print the structure of the first element
FAQ[0]

{'question': 'What are your store hours?',
 'answer': 'Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.',
 'type': 'general information'}

<a id='ex04'></a>

#### 3.2.1 Creating the FAQ Layout

Now we will generate the FAQ layout as discussed above.

The FAQ Layout will be the following:

```
Question: FAQ Question 1, Answer: FAQ Answer 1, Type: FAQ Type 1
...
Question: FAQ Question 25, Answer: FAQ Answer 25, Type: FAQ Type 25
```

In [19]:
def generate_faq_layout(faq_dict: list) -> str:
    """
    Generates a formatted string layout for a list of FAQs.

    This function iterates through a dictionary of frequently asked questions (FAQs) and constructs
    a string where each question is followed by its corresponding answer and type.

    Parameters:
    - faq_dict (list): A list of dictionaries, each containing keys 'question', 'answer', and 'type' 
      representing an FAQ entry.

    Returns:
    - str: A string representing the formatted layout of FAQs, with each entry on a separate line.
    """
    # Initialize an empty string
    t = ""

    # Iterate over every FAQ question in the FAQ list
    for f in faq_dict:
        # Append the question with formatted string (remember to use f-string and access the values as f['question'], f['answer'] and so on)
        # Also, do not forget to add a new line character (\n) at the end of each line.
        t += f"Question: {f['question']} Answer: {f['answer']} Type: {f['type']}\n" 
  

    return t

In [20]:
FAQ_LAYOUT = generate_faq_layout(FAQ)
print(FAQ_LAYOUT[:1000])

Question: What are your store hours? Answer: Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday. Type: general information
Question: Where is Fashion Forward Hub located? Answer: Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State. Type: general information
Question: Do you have a physical store location? Answer: At this time, we operate exclusively online. This allows us to offer a broader selection and lower prices directly to you. Type: general information
Question: How can I create an account with Fashion Forward Hub? Answer: Click on 'Sign Up' in the top right corner of our website and follow the instructions to set up your account. Type: general information
Question: How do I subscribe to your newsletter? Answer: To receive the latest updates and promotions, sign up for our newsletter at the bottom of our homepage. Type: general information
Question:

<a id='ex02'></a>

---
Great! Now that we have the FAQ layout ready, our next task is to create a function that answers questions based on the FAQ content. we’ll inject the FAQ layout into a new, hardcoded prompt. one common approach is to wrap the FAQ layout using the `<FAQ> </FAQ>` tags.

In [21]:

def query_on_faq(query: str, **kwargs) -> dict:
    """
    Constructs a prompt to query an FAQ system and generates a response.

    Parameters:
    - query (str): The query about which the function seeks to provide an answer from the FAQ.
    - **kwargs: Optional keyword arguments for extra configuration of prompt parameters.

    Returns:
    - str: The response generated from the LLM based on the input query and FAQ layout.

    """
    ### START CODE HERE ###

    # Make the prompt. Don't forget to add the FAQ_LAYOUT and the query in it!
    prompt = f'''
    You will be provided with an FAQ for a clothing store.
    Answer the user's question using only the information in the FAQ. 
    If needed, combine multiple relevant FAQ entries. 
    Do NOT mention that you are using an FAQ. 
    If the FAQ does not contain the answer, say: "I'm not sure based on our FAQ."
    
    <FAQ>
    PROVIDED FAQ: {FAQ_LAYOUT}
    </FAQ>
    Question: {query}
    Answer:
    '''.strip()
    

    # Generate the parameters dict with PROMPT and **kwargs 
    kwargs = generate_params_dict(prompt, **kwargs)

    ### END CODE HERE ###
    
    return kwargs

In [22]:
kwargs = query_on_faq("Are return shipping costs covered? ?")
kwargs

{'prompt': 'You will be provided with an FAQ for a clothing store.\n    Answer the user\'s question using only the information in the FAQ. \n    If needed, combine multiple relevant FAQ entries. \n    Do NOT mention that you are using an FAQ. \n    If the FAQ does not contain the answer, say: "I\'m not sure based on our FAQ."\n    \n    <FAQ>\n    PROVIDED FAQ: Question: What are your store hours? Answer: Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday. Type: general information\nQuestion: Where is Fashion Forward Hub located? Answer: Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State. Type: general information\nQuestion: Do you have a physical store location? Answer: At this time, we operate exclusively online. This allows us to offer a broader selection and lower prices directly to you. Type: general information\nQuestion: How can I create an account

In [23]:
content = generate_with_single_input(**kwargs)

In [24]:
print(content['content'])

We provide a prepaid return label for domestic returns. For international returns, shipping is at the customer's cost.



### 3.3 Decide the Nature of a Product-Related Question

Now, let's start working with product-related queries.

<a id='ex03'></a>
### Exercise 3
---
Fashion Forward only wants the chatbot to answer the following types of queries:

- **Technical queries** – asking for descriptions of specific products, such as whether a blue dress is available or requesting three examples of red T-shirts suitable for sunny days.
- **Creative queries** – asking for help creating a stylish look for visiting a museum.

We will reate a prompt with clear instructions (and examples!) alongside the query. Remember to ensure the model only outputs **"creative"** or **"technical"** by:

- setting a lower temperature 
- and explicitly stating this in the prompt.


Decide if the following query is a query that requires creativity (creating, composing, making new things) or technical (information about products, prices, etc.). Label it as creative or technical.
              Examples:
              Give me suggestions on a nice look for a nightclub. Label: creative
              What are the blue dresses you have available? Label: technical
              Give me three T-shirts for summer. Label: technical
              Give me a look for attending a wedding party. Label: creative
              Query to be analyzed: {query}. Only output one token: the label.
    </code></pre>
</details>



In [25]:


def decide_task_nature(query: str) -> str:
    """
    Determines whether a query is creative or technical.

    This function constructs a prompt for an LLM to decide if a given query requires a creative response,
    such as making suggestions or composing ideas, or a technical response, such as providing product details or prices.

    Parameters:
    - query (str): The query to be evaluated for its nature.

    Returns:
    - str: The label 'creative' if the query requires creative input, or 'technical' if it requires technical information.
    """

    ### START CODE HERE ###

    # Create the prompt. Remember to include the query, examples, and clear instructions (not necessarily in this order!)
    prompt = f"""
    Decide if the following query is a query that requires creativity (creating, composing, making new things) or technical (information about products, prices, etc.). Label it as creative or technical.
    Examples:
    Give me suggestions on a nice look for a nightclub. Label: creative
    What are the blue dresses you have available? Label: technical
    Give me three T-shirts for summer. Label: technical
    Give me a look for attending a wedding party. Label: creative

    Instructions: Query to be analyzed: {query}. Only output one token: the label..
    """

    # Generate the kwargs dictionary by passing the PROMPT, setting temperature to 0 and max_tokens to 1
    kwargs = generate_params_dict(prompt, temperature=0, max_tokens=1)

    # Generate the response using generate_with_single_input and **kwargs
    response = generate_with_single_input(**kwargs)

    # Get the label
    label = response['content']

    ### END CODE HERE ###
    
    return label

In [26]:
queries = ["Give me two sneakers with vibrant colors.",
           "What are the most expensive clothes you have in your catalogue?",
           "I have a green dress and I like a suggestion on an accessory to match with it.",
           "Give me three trousers with vibrant colors you have in your catalogue.",
           "Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather."
           ]

In [27]:
for query in queries:
    label = decide_task_nature(query)
    print(f"Query: {query} Label: {label}")

Query: Give me two sneakers with vibrant colors. Label: technical
Query: What are the most expensive clothes you have in your catalogue? Label: technical
Query: I have a green dress and I like a suggestion on an accessory to match with it. Label: creative
Query: Give me three trousers with vibrant colors you have in your catalogue. Label: technical
Query: Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather. Label: creative


<a id='3-4'></a>
### 3.4 Retrieving the Parameters for a Given Task

We will create a function that, given a task, returns the appropriate values for `top_p` and `temperature`.

For **technical** queries, **low randomness is preferred**, whereas for **creative** tasks, **higher randomness might be more suitable**. 

**Important:** If the task is neither `technical` nor `creative` (for example, if the LLM fails to output a valid label), then fallback to a default set of parameters. You can decide whether to choose a middle ground between low and high randomness or stick to low randomness as a conservative approach.

**Note**: Remember that a temperature that is too high will lead the model to nonsense results so keep it below **1.3** and if it is close to **1.3**, be sure to lower the **top_p**. Also, remember that *top_p* cannot be greater than 1!

for Gemini models specificly the temperature can be set from 0.0 to 2.0 (inclusive)


In [28]:

def get_params_for_task(task: str) -> dict:
    """
    Retrieves specific LLM parameters based on the nature of the task.

    This function returns parameter sets optimized for either creative or technical tasks.
    Creative tasks benefit from higher randomness, while technical tasks require more focus and precision.
    A default parameter set is returned for unrecognized task types.

    Parameters:
    - task (str): The nature of the task ('creative' or 'technical').

    Returns:
    - dict: A dictionary containing 'top_p' and 'temperature' settings appropriate for the task.
    """
    ### START CODE HERE ###
    # Define the parameter sets for technical and creative tasks
    PARAMETERS_DICT = {
        "creative": {"top_p": 0.8, 'temperature': 1.0},
        "technical": {'top_p': 0.3, 'temperature': 0.3}
    }
    
    # Return the corresponding parameter set based on task type
    if task == 'technical':
        param_dict = PARAMETERS_DICT['technical']
    elif task == 'creative':
        param_dict = PARAMETERS_DICT['creative']
    else:
        # Fallback to a default parameter set for unrecognized task types
        param_dict = {"top_p": 0.6, "temperature": 0.7}
    ### END CODE HERE ###
    
    return param_dict


In [29]:
get_params_for_task("technical")

{'top_p': 0.3, 'temperature': 0.3}

<a id='4'></a>
## 4 - Retrieving Items Based on Metadata Inferred from a Query
---
In this section, we’ll create a function to extract useful metadata to help filter the items shown to it. we’ll get a JSON file with different features and all the possible values found in the dataset. The job is to pass these values to the database, so the LLM can pick the ones that make the most sense. And of course, we'll also need to handle situations where the LLM might not find a correct value.

The values we’ll focus on are:
- gender  
- masterCategory  
- articleType  
- baseColour  
- season  
- usage

These were chosen because they strike a good balance — they’re specific enough to be useful, but general enough to avoid empty results. Some other features in the dataset are too detailed and could lead to no matches. Also, including every single value would make the prompt too large, which could slow things down and raise costs — something to keep in mind when building real-world solutions!


In [30]:
# Let's remember the data structure of a product
PRODUCTS_DATA[0]

{'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Shirts',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011.0,
 'usage': 'Casual',
 'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
 'price': 67,
 'product_id': 15970}

In [31]:
# Run this cell to generate the dictionary with the possible values for each key
values = {}
for d in PRODUCTS_DATA:
    for key, val in d.items():
        if key in ('product_id', 'price', 'productDisplayName', 'subCategory', 'year'):
            continue
        if key not in values.keys():
            values[key] = set()
        values[key].add(val)

In [32]:
# Example of possible values for the feature 'season'
values['season']

{'All seasons', 'Fall', 'Spring', 'Summer', 'Winter'}

In [33]:
values.keys()

dict_keys(['gender', 'masterCategory', 'articleType', 'baseColour', 'season', 'usage'])

<a id='5-1'></a>

<a id='4-1'></a>
### 4.1 Generate metadata


The next function’s purpose is to extract potential metadata from a given query. The approach is to construct a prompt that incorporates the `values` dictionary, which lists possible feature values. the LLM is then asked to generate a JSON response suggesting metadata relevant to the query. You have the flexibility to add more information as needed.

In addition to the metadata features, the LLM must also handle price constraints. If the query specifies a price range, the JSON should include a key like this:

```json
"price": {"min": min_value, "max": max_value}
```

If no price constraint is provided, the LLM should default to:

```json
"price": {"min": 0, "max": "inf"}
```

Here is an example of the expected JSON format you should explicitly include in your prompt to help guide the LLM:

```json
{
    "gender": ["Women"],
    "masterCategory": ["Apparel"],
    "articleType": ["Dresses"],
    "baseColour": ["Blue"],
    "price": {"min": 0, "max": "inf"},
    "usage": ["Formal"],
    "season": ["All seasons"]
}
```

**Important Note**: When using f-strings in Python, you must use double curly braces within the string to ensure it is parsed as a literal. For example:

```python
f"""Any text here {{
    "gender": ["Women"],
    "masterCategory": ["Apparel"],
    "articleType": ["Dresses"],
    "baseColour": ["Blue"],
    "price": {{"min": 0, "max": "inf"}},
    "usage": ["Formal"],
    "season": ["All seasons"]
}}"""
```

**Note**: To avoid truncating a JSON, set the `max_tokens` value for something around `1500`!

We use double curly braces within an f-string to ensure that Python interprets them correctly as part of the string.


In [34]:
def generate_metadata_from_query(query: str) -> str:
    """
    Generates metadata in JSON format based on a given query to filter clothing items.
    """

    # Define possible values
    values = {
        "gender": ["Men", "Women", "Unisex"],
        "masterCategory": ["Apparel", "Accessories", "Footwear"],
        "articleType": ["Tshirts", "Shirts", "Dresses", "Jeans", "Shorts", "Shoes", "Bags", "Trousers"],
        "baseColour": ["Black", "White", "Blue", "Red", "Green", "Yellow", "Grey", "Beige", "Pink", "Navy Blue"],
        "usage": ["Casual", "Formal", "Sports", "Ethnic", "Party"],
        "season": ["Summer", "Winter", "Spring", "Autumn", "All Seasons"]
    }

    # Construct prompt
    prompt = f"""
    A query will be provided. Based on this query, a vector database will be searched to find relevant clothing items.
    Generate a JSON object containing useful metadata to filter products for this query.
    The possible values for each feature are given in the following JSON: {values}
    You must output ONLY valid JSON (no code fences, no extra text).
    Return keys: gender, masterCategory, articleType, baseColour, price, usage, season.
    All keys must be present, but only once.
    All values except 'price' are lists of strings from this set:
    {json.dumps(values, ensure_ascii=False)}
    'price' is an object: {{"min": <number or 0>, "max": <number or "inf">}}.
    If no price mentioned, use min=0 and max="inf".

    Example of expected JSON:

    {{
      "gender": ["Women"],
      "masterCategory": ["Apparel"],
      "articleType": ["Dresses"],
      "baseColour": ["Blue"],
      "price": {{"min": 0, "max": "inf"}},
      "usage": ["Formal"],
      "season": ["All Seasons"]
    }}

    Query: {query}
    """.strip()
    

    # Call LLM (Gemini)
    response = generate_with_single_input(
    prompt=prompt,
    temperature=0,
    max_tokens=1500,
    mime_type="application/json")  # ✅ Enforce JSON

    # Extract the model's text output
    content = response["content"]
    metadata = json.loads(content)

    return metadata

In [35]:
generate_metadata_from_query("Create a look for a man that suits a sunny day in the park. I don't want to spend more than 300 dollars on each piece.")

{'gender': ['Men'],
 'masterCategory': ['Apparel', 'Accessories', 'Footwear'],
 'articleType': ['Tshirts', 'Shirts', 'Shorts', 'Jeans', 'Shoes'],
 'baseColour': ['Black',
  'White',
  'Blue',
  'Red',
  'Green',
  'Yellow',
  'Grey',
  'Beige',
  'Pink',
  'Navy Blue'],
 'price': {'min': 0, 'max': 300},
 'usage': ['Casual'],
 'season': ['Summer', 'Spring']}

The next functions are helper functions to extract the JSON from the query. You also need to handle the case where the LLM doesn't provide a valid and recoverable JSON. In this case, the code will just create an empty filter.

In [36]:
def parse_json_output(llm_output: str) -> dict:
    """
    Parses a string output from an LLM into a JSON object.

    This function attempts to clean and parse a JSON-formatted string produced by an LLM.
    The input string might contain minor formatting issues, such as unnecessary newlines or single quotes
    instead of double quotes. The function attempts to correct such issues before parsing.

    Parameters:
    - llm_output (str): The string output from the LLM that is expected to be in JSON format.

    Returns:
    - dict or None: A dictionary if parsing is successful, or None if the input string cannot be parsed into valid JSON.

    Exception Handling:
    - In case of a JSONDecodeError during parsing, an error message is printed, and the function returns None.
    """
    try:
        # Since the input might be improperly formatted, ensure any single quotes are removed
        llm_output = llm_output.replace("\n", "").replace("'", '"').replace("}}", "}").replace("{{", "{")  # Remove any erroneous structures
        
        # Attempt to parse JSON directly provided it is a properly-structured JSON string
        parsed_json = json.loads(llm_output)
        return parsed_json
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        return None

In [37]:
json_string = generate_metadata_from_query("Give me three blue dresses suitable for a wedding party, less than 200 dollars and at least 50 dollars")
print(json_string)


{'gender': ['Women'], 'masterCategory': ['Apparel'], 'articleType': ['Dresses'], 'baseColour': ['Blue'], 'price': {'min': 50, 'max': 200}, 'usage': ['Party', 'Formal'], 'season': ['All Seasons']}


**TIP**: Try with different queries and check if the JSON is properly parsed. If not, investigate why and maybe improve the PROMPT to avoid such issue.

In [38]:
# Drop & recreate
client = weaviate.connect_to_local(host="localhost", port=8080, grpc_port=50051)



In [39]:
products_collection = client.collections.get("Product1")
len(products_collection)

36850

<a id='4-3'></a>
### 4.3 Filtering by metadata 
This next function will create the filters given the metadata. It will create a `Filter` object for each key in the dictionary of metadata. 

In [40]:
def embed_query(text: str) -> list[float]:
    # MUST match your ingestion embedder
    return _embed_utils(text) if EMBEDDINGS_BACKEND == "utils" else _embed_http(text)

In [41]:
# --- Helper: build a map {key: [Filter,...]} so we can drop by key cleanly
def get_filter_map_by_metadata(json_output: dict | None = None) -> dict[str, list]:
    if not json_output:
        return {}

    valid_keys = ("gender", "masterCategory", "articleType", "baseColour", "price", "usage", "season")
    fmap: dict[str, list] = {}

    for key in valid_keys:
        if key not in json_output:
            continue
        value = json_output[key]

        if key == "price":
            if isinstance(value, dict):
                min_price = value.get("min", None)
                max_price = value.get("max", None)
                items = []
                if isinstance(min_price, (int, float)) and min_price > 0:
                    items.append(Filter.by_property("price").greater_than(min_price))
                if isinstance(max_price, (int, float)):
                    items.append(Filter.by_property("price").less_than(max_price))
                if items:
                    fmap["price"] = items
            continue

        if value is None:
            continue
        if not isinstance(value, list):
            value = [value]
        cleaned = [v for v in value if isinstance(v, str) and v.strip() and v != "Any"]
        if cleaned:
            fmap[key] = [Filter.by_property(key).contains_any(cleaned)]

    return fmap

def _make_all_of(filter_map: dict[str, list], include_keys) -> "Filter | None":
    selected = []
    for k in include_keys:
        selected.extend(filter_map.get(k, []))
    return Filter.all_of(selected) if selected else None

This is wrapper function, that, given a query, return the desired filters.

In [42]:
def generate_filters_from_query(query: str) -> list:
    json_output = generate_metadata_from_query(query)
    filters = get_filter_map_by_metadata(json_output)
    return filters

In [43]:
generate_metadata_from_query("Give me three T-shirts to use in sunny days")

{'gender': [],
 'masterCategory': ['Apparel'],
 'articleType': ['Tshirts'],
 'baseColour': [],
 'price': {'min': 0, 'max': 'inf'},
 'usage': ['Casual'],
 'season': ['Summer']}

In [44]:
filters = generate_filters_from_query("Give me three T-shirts to use in sunny days")

In [45]:
filters

{'masterCategory': [_FilterValue(value=['Apparel'], operator=<_Operator.CONTAINS_ANY: 'ContainsAny'>, target='masterCategory')],
 'articleType': [_FilterValue(value=['Tshirts'], operator=<_Operator.CONTAINS_ANY: 'ContainsAny'>, target='articleType')],
 'usage': [_FilterValue(value=['Casual'], operator=<_Operator.CONTAINS_ANY: 'ContainsAny'>, target='usage')],
 'season': [_FilterValue(value=['Summer'], operator=<_Operator.CONTAINS_ANY: 'ContainsAny'>, target='season')]}

Note that the filters are there with the correct metadata.

The next function will get the relevant products from the query, by generating the filters, running a semantic search using the query, and then perform the metadata filtering to narrow down the possibilities and increase accuracy. 

It deals with the case where the set of metadata returns too few results by incrementally removing some filters until it gets a result with more than 5 possibilities.

In [46]:
# --- Main function: semantic search + metadata filtering with graceful relaxation
def get_relevant_products_from_query(query: str, *, want_min_results: int = 5, target_vector: str | None = None):
    # 1) metadata → filters
    meta = generate_metadata_from_query(query)
    f_map = get_filter_map_by_metadata(meta)
    have_keys = list(f_map.keys())

    # 2) relaxation order (specific → general)
    preferred_keep_order = ["articleType", "baseColour", "usage", "season", "gender", "masterCategory", "price"]
    keep_order = [k for k in preferred_keep_order if k in have_keys]

    # 3) price introspection (only if your Filter exposes operator name)
    price_filters = f_map.get("price", [])
    def _has_op(filters, names):
        return any(getattr(f, "operator", None) in names for f in filters)
    has_price_lower = _has_op(price_filters, ("GreaterThan", "GreaterThanEqual"))
    has_price_upper = _has_op(price_filters, ("LessThan", "LessThanEqual"))

    # 4) choose search primitive
    use_semantic = True
    try:
        qvec = embed_query(query)
        if not isinstance(qvec, (list, tuple)) or not qvec:
            use_semantic = False
    except Exception:
        use_semantic = False

    def _search(include_keys):
        flt = _make_all_of(f_map, include_keys)
        if use_semantic:
            return products_collection.query.near_vector(
                near_vector=qvec,
                filters=flt,
                limit=20,
                target_vector=target_vector,
            ).objects
        else:
            return products_collection.query.bm25(
                query=query,
                filters=flt,
                limit=20,
            ).objects

    # 5) strict run
    res = _search(keep_order)
    if len(res) >= max(10, want_min_results):
        return res

    # 6) relax price first
    if "price" in keep_order and (has_price_upper or has_price_lower):
        orig = list(price_filters)

        # drop only upper bound
        if has_price_upper:
            f_map["price"] = [f for f in orig if getattr(f, "operator", None) in ("GreaterThan", "GreaterThanEqual")]
            res = _search(keep_order)
            if len(res) >= want_min_results:
                return res
            f_map["price"] = list(orig)

        # drop only lower bound
        if has_price_lower:
            f_map["price"] = [f for f in orig if getattr(f, "operator", None) in ("LessThan", "LessThanEqual")]
            res = _search(keep_order)
            if len(res) >= want_min_results:
                return res
            f_map["price"] = list(orig)

        # drop price entirely
        keep_order = [k for k in keep_order if k != "price"]
        res = _search(keep_order)
        if len(res) >= want_min_results:
            return res

    # 7) drop categorical one by one
    for k in ["articleType", "baseColour", "usage", "season", "gender", "masterCategory"]:
        if k in keep_order:
            keep_order = [x for x in keep_order if x != k]
            res = _search(keep_order)
            if len(res) >= want_min_results:
                return res

    # 8) final fallback: no filters
    if use_semantic:
        return products_collection.query.near_vector(near_vector=qvec, limit=20, target_vector=target_vector).objects
    else:
        return products_collection.query.bm25(query=query, limit=20).objects

In [47]:
get_relevant_products_from_query("Give me three T-shirts to use in sunny days")

[Object(uuid=_WeaviateUUIDInt('22275657-4ab7-50e2-a0fa-e995b8601979'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2016.0, 'masterCategory': 'Apparel', 'subCategory': 'Topwear', 'productDisplayName': '2go Active Gear USA Men Pack of Three T-shirts', 'product_id': 27388.0, 'price': 115.0, 'gender': 'Men', 'articleType': 'Tshirts', 'usage': 'Casual', 'baseColour': 'Black', 'season': 'Summer'}, references=None, vector={}, collection='Product1'),
 Object(uuid=_WeaviateUUIDInt('80d563ed-b87b-507d-aed7-193bd0a29117'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2011.0, 'product_id': 3423.0, 'subCategory': 'Topwear', 'usage': 'Casual', 'masterCategory': 'Apparel', 'productDisplayName': "Myntra Women's I On

In [48]:
products_collection.query.bm25("summer dress", limit=20).objects

[Object(uuid=_WeaviateUUIDInt('f9642077-a5e9-5096-a27f-24bc09088039'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2012.0, 'masterCategory': 'Apparel', 'price': 23.0, 'usage': 'Casual', 'subCategory': 'Dress', 'product_id': 49631.0, 'gender': 'Women', 'articleType': 'Dresses', 'productDisplayName': 'Mineral Cream Dress', 'baseColour': 'Cream', 'season': 'Summer'}, references=None, vector={}, collection='Product1'),
 Object(uuid=_WeaviateUUIDInt('a0d67a9c-329b-550b-a37a-eec089936213'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'year': 2012.0, 'product_id': 59990.0, 'price': 138.0, 'productDisplayName': 'Avirate Pink Dress', 'masterCategory': 'Apparel', 'subCategory': 'Dress', 'gender': 'Women', 'articleTyp

In [49]:
t = get_relevant_products_from_query("Give me three T-shirts to use in sunny days")

In [50]:
# Check if t is non-empty
if len(t) > 0:
    print(t[0].properties)

{'year': 2016.0, 'masterCategory': 'Apparel', 'subCategory': 'Topwear', 'usage': 'Casual', 'productDisplayName': '2go Active Gear USA Men Pack of Three T-shirts', 'price': 115.0, 'gender': 'Men', 'articleType': 'Tshirts', 'product_id': 27388.0, 'baseColour': 'Black', 'season': 'Summer'}


So, one of the relevant results is indeed a Tshirt! 

<a id='4-4'></a>
### 4.4 Generating the retrieve items as a context (NOT GRADED)

Now, for the given retrieved items, let's generate a simple context in the format 

```
Product name: Inkfruit Mens Little Bit More T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Tshirts. Product Category: Topwear Product Color: Yellow. Product Season: Summer. Product Year: 2011.
```

In [51]:
def generate_items_context(results: list) -> str:
    """
    Compile detailed product information from a list of result objects into a formatted string.

    This function takes a list of results, each containing various product attributes, and constructs 
    a human-readable summary for each product. Each product's details, including ID, name, category, 
    usage, gender, type, and other characteristics, are concatenated into a string that describes 
    all products in the list.

    Parameters:
    results (list): A list of result objects, each having a `properties` attribute that is a dictionary 
                    containing product attributes such as 'product_id', 'productDisplayName', 
                    'masterCategory', 'usage', 'gender', 'articleType', 'subCategory', 
                    'baseColour', 'season', and 'year'.

    Returns:
    str: A multi-line string where each line contains the formatted details of a single product.
         Each product detail includes the product ID, name, category, usage, gender, type, color, 
         season, and year.
    """
    t = ""  # Initialize an empty string to accumulate product information

    for item in results:  # Iterate through each item in the results list
        item = item.properties  # Access the properties dictionary of the current item

        # Append formatted product details to the output string
        t += (
            f"Product ID: {item['product_id']}. "
            f"Product name: {item['productDisplayName']}. "
            f"Product Category: {item['masterCategory']}. "
            f"Product usage: {item['usage']}. "
            f"Product gender: {item['gender']}. "
            f"Product Type: {item['articleType']}. "
            f"Product Category: {item['subCategory']} "
            f"Product Color: {item['baseColour']}. "
            f"Product Season: {item['season']}. "
            f"Product Year: {item['year']}.\n"
        )

    return t  # Return the complete formatted string with product details

In [52]:
print(generate_items_context(t)[:1000])

Product ID: 27388.0. Product name: 2go Active Gear USA Men Pack of Three T-shirts. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Tshirts. Product Category: Topwear Product Color: Black. Product Season: Summer. Product Year: 2016.0.
Product ID: 3423.0. Product name: Myntra Women's I Only Give Negative Feedback Black T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Women. Product Type: Tshirts. Product Category: Topwear Product Color: Black. Product Season: Summer. Product Year: 2011.0.
Product ID: 3354.0. Product name: Myntra Men's I Give 100 Percent Work White T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Tshirts. Product Category: Topwear Product Color: White. Product Season: Summer. Product Year: 2011.0.
Product ID: 3356.0. Product name: Myntra Men's I Give 100 Percent Work Black T-shirt. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: 

<a id='4-5'></a>
### 4.5 Query on Products (NOT GRADED)

You’re almost there! This section explains the function that queries products based on a given task. The process follows these steps:

1. **Query**: Start with a product query.
2. **Determine Query Nature**: Identify if the query is technical or creative.
3. **Retrieve Relevant Products**: Find products that best match the query criteria.
4. **Generate Context**: Build a descriptive context string based on the products.
5. **Create Prompt**: Formulate the prompt using the context and the query nature.
6. **Generate Parameters**: Prepare parameters suited to the query nature for the LLM.
7. **Run Inference**: Perform the inference using the prepared parameters.

Let's check the code!


In [53]:
def query_on_products(query: str) -> dict:
    """
    Execute a product query process to generate a response based on the nature of the query.

    This function analyzes the type of query — whether it is technical or creative — and retrieves 
    relevant product information accordingly. It constructs a prompt that includes product details 
    and the original query, and then generates parameters for querying an LLM.
    Finally, it generates a response based on the prompt and returns the content of the response.

    Parameters:
    query (str): The input query string that needs to be analyzed and answered using product data.

    Returns:
    dict: A dictionary of keyword arguments (`kwargs`) containing the prompt and additional settings 
          for creating a response, suitable for input to an LLM or other processing system.

    Outputs:
    dict: A dictionary with the parameters to call an LLM
    """


    # Determine if the query is technical or creative in nature
    query_label = decide_task_nature(query) 
    
    # Obtain necessary parameters based on the query type
    parameters_dict = get_params_for_task(query_label) 
    
    # Retrieve products that are relevant to the query
    relevant_products = get_relevant_products_from_query(query) 
     
    # Create a context string from the relevant products
    context = generate_items_context(relevant_products) 

    # Construct a prompt including product details and the query. Remember to add the context and the query in the prompt, also, ask the LLM to provide the product ID in the answer
    prompt = (
    f"Given the available set of cloth products, answer the question that follows, providing the item ID in your answers. "
    f"Other information might be provided but not necessarily all of them; pick only the relevant ones for the given query and avoid being too long when describing the items' features. "
    f"If no number of products is mentioned in the query, select at most five to show. "
    f"CLOTH PRODUCTS AVAILABLE: {context} "
    f"QUERY: {query}"
        )
    
    # Generate kwargs (parameters dict) for parameterized input to the LLM with , Prompt, role = 'assistant' and **parameters_dict
    kwargs = generate_params_dict(prompt, role='assistant', **parameters_dict)
    
    
    return kwargs

In [54]:
kwargs = query_on_products('Make a wonderful look for a man attending a wedding party happening during night.')

In [55]:
result = generate_with_single_input(**kwargs)
print(result['content'])

Here are a few options for a wedding party look for a man:

*   **Product ID: 56855.0.** John Players Men Blue Shirt, perfect for a party.
*   **Product ID: 39381.0.** Peter England Men Party Black Jeans, ideal for a casual yet stylish look.
*   **Product ID: 21059.0.** Clarks Men Extra Look Black Formal Shoe, appropriate footwear for the event.
*   **Product ID: 14166.0.** Belmonte Men Snake Skin Look Black Belts, a great accessory for a formal occasion.


In [56]:
kwargs = query_on_products('Give me three T-shirts for sunny days')

In [57]:
result = generate_with_single_input(**kwargs)
print(result['content'])

1. Product ID: 27388.0 - 2go Active Gear USA Men Pack of Three T-shirts. These T-shirts are suitable for casual wear during the summer season.
2. Product ID: 3423.0 - Myntra Women's I Only Give Negative Feedback Black T-shirt. This black T-shirt is designed for casual wear during the summer.
3. Product ID: 3354.0 - Myntra Men's I Give 100 Percent Work White T-shirt. This white T-shirt is designed for casual wear during the summer.



<a id='5'></a>
## 5 - The Final Function!

---

<a id='5-1'></a>
### 5.1 The function to rule them all

Now it’s time to bring everything together into a single function!

This function will:

1. Check if the query is related to an FAQ or a Product.
2. If it’s an FAQ, run the FAQ-related workflow.
3. If it’s a Product, run the Product-related workflow.

It returns the kwargs dictionary containing the appropriate arguments.

In [58]:
def answer_query(query: str) -> dict:
    """
    Determines the type of a given query (FAQ or Product) and executes the appropriate workflow.

    Parameters:
    - query (str): The user's query string.

    Returns:
    - dict: A dictionary of keyword arguments to be used for further processing.
      If the query is neither FAQ nor Product-related, returns a default response dictionary
      instructing the assistant to answer based on existing context.
    """
    label = check_if_faq_or_product(query)
    if label not in ['FAQ', 'Product']:
        return {
            "role": "assistant",
            "prompt": f"User provided a question that does not fit FAQ or Product related questions. "
                      f"Answer it based on the context you already have so far. Query provided by the user: {query}"
        }
    if label == 'FAQ':
        kwargs = query_on_faq(query)
    if label == 'Product':
        try:
            kwargs = query_on_products(query)
        except:
            return {
            "role": "assistant",
            "prompt": f"User provided a question that broke the querying system. Instruct them to rephrase it."
                      f"Answer it based on the context you already have so far. Query provided by the user: {query}"
        }
            
    return kwargs

In [59]:
kwargs = answer_query("What are your working hours?")

In [60]:
result = generate_with_single_input(**kwargs)
print(result['content'])

Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.



In [61]:
kwargs = answer_query("give me some nice ladies shoes for a hot summer wedding?")
result = generate_with_single_input(**kwargs)
print(result['content'])

Given the context of a hot summer wedding, here are a few options for women's shoes:

*   **Product ID: 6173.0.** ADIDAS Women's Find Me White Flip Flop. This is a white flip flop.
*   **Product ID: 35784.0.** Enroute Women Black Shoes. These are black flats.
*   **Product ID: 35800.0.** Enroute Women Brown Shoes. These are brown flats.
*   **Product ID: 43367.0.** Reebok Women Black Casual Shoes. These are black flats.
*   **Product ID: 20735.0.** Enroute Women Mocassin Red Shoes. These are red flats.


<a id='5-2'></a>
### 5.2 The ChatBot

Now you can implement the ChatBot! It is already given to you, as it is not the focus of this course or assignment, but feel free to inspect the utils.py file to understand how it works (and improve it as you wish!)

Suggested queries:

- Do you have blue t-shirts on your catalogue?
- I bought a dress and I didn't like it. How can I get a refund?
- I am going to a party at the beach. Can you suggest a nice look for me? It will be a warm night, and I’m a man.

In [62]:
kwargs = answer_query("Do you have blue t-shirts on your catalogue??")
result = generate_with_single_input(**kwargs)
print(result['content'])

Yes, here are some blue t-shirts from the catalogue:
* Product ID: 34364.0. Myntra Men Blue Fuel Your Desire T-shirt. For men, casual usage.
* Product ID: 28571.0. Nike Women You Wish Blue T-shirt. For women, casual usage.
* Product ID: 34361.0. Myntra Men Blue Do Bananas T-shirt. For men, casual usage.
* Product ID: 3265.0. Guerrilla Men's Kill You Blue T-shirt. For men, casual usage.
* Product ID: 54930.0. Do u speak green Men Blue T-shirt. For men, casual usage.


In [63]:
kwargs = answer_query("Do you have blue t-shirts on your catalogue?")

# Build the OpenAI-style messages list the helper expects
messages = [{
    "role": kwargs.get("role", "user"),
    "content": kwargs["prompt"]
}]

# Pass messages positionally, and the rest as kwargs
result = generate_with_multiple_input(
    messages,
    top_p=kwargs.get("top_p"),
    temperature=kwargs.get("temperature"),
    max_tokens=kwargs.get("max_tokens", 500),
    model=kwargs.get("model")
)
print(result["content"])

Yes, here are some blue t-shirts:
- Myntra Men Blue Fuel Your Desire T-shirt (Product ID: 34364.0), a blue t-shirt for men, designed for casual usage in the summer of 2012.
- Nike Women You Wish Blue T-shirt (Product ID: 28571.0), a blue t-shirt for women, designed for casual usage in the summer of 2012.
- Myntra Men Blue Do Bananas T-shirt (Product ID: 34361.0), a blue t-shirt for men, designed for casual usage in the summer of 2012.
- Guerrilla Men's Kill You Blue T-shirt (Product ID: 3265.0), a blue t-shirt for men, designed for casual usage in the summer of 2011.
- Do u speak green Men Blue T-shirt (Product ID: 54930.0), a blue t-shirt for men, designed for casual usage in the summer of 2012.



## 6. Putting It All Together: RAG Chatbot Flow

The final chatbot pipeline uses all the pieces above:

1. **Routing** – `check_if_faq_or_product(query)` decides between FAQ and Product.
2. **Conditional parameters** – `generate_params_dict(...)` selects suitable `temperature`, `top_p`, and `max_tokens`.
3. **JSON metadata** (Product path only) –  
   `generate_metadata_from_query(query)` → LLM returns a JSON object with filters (gender, category, color, price, etc.).  
   `parse_json_output(...)` → clean & parse into a Python dict.
4. **Retrieval** – metadata filters + embeddings are used to query Weaviate for the most relevant products.
5. **Final LLM answer** – `generate_with_multiple_input(...)` or `call_llm_with_context(...)` generates the answer, grounded in retrieved products and/or FAQs.
6. **Chat UI** – the ChatWidget or Flask app calls this logic and displays the answer to the user.

This demonstrates a full **RAG + routing + JSON outputs** design, not just a simple “ask the LLM” demo.


In [74]:
from utils import ChatWidget
chat = ChatWidget(generator_function=answer_query)

---

## 👤 Author Note

This notebook was originally inspired by the “Retrieval Augmented Generation (RAG)” Coursera course, which provided the foundational framework for routing, metadata extraction, and the general RAG workflow.

However, this project reflects my **own engineering work and extensions**, including:

- Fully migrating the assignment to the **Google Gemini API** for both embeddings and text generation  
- Rebuilding the entire embedding and retrieval pipeline to run on a **local Weaviate instance deployed via Docker**  
- Creating and managing the Weaviate schema, ingestion pipeline, and vectorization inside my **own development environment**  
- Implementing the complete RAG flow locally:  
  - generating Gemini embeddings  
  - inserting thousands of product documents  
  - performing hybrid search and reranking  
  - building JSON-based metadata filters  
  - integrating all components into a chatbot workflow  
- Adapting prompt design, JSON parsing, conditional parameters, and routing logic specifically for Gemini models  
- Ensuring the notebook is fully **self-contained and reproducible** outside the Coursera infrastructure

In summary, while the original course provided the conceptual structure, all system integration, Gemini migration, Docker setup, Weaviate management, and local execution represent my own extensions and hands-on implementation.

**— Juan Zhang**
